In [13]:
#unzips and re-zips individual log files and processes each one by calling out to the single file player 
#log processor
import logging
import os
import pdb
import tarfile
from bs4 import BeautifulSoup
import re
import shutil
import time
import math
import pandas as pd

In [27]:
#set name of input file that contains a list of logs to build play traces from
input_filename = 'ResultsFiles/2Player_FG1E_Supply_AllYears_logfilenames.txt'
output_filename = 'ResultsFiles/2Player_FG1E_Supply_AllYears_Traces.txt'

#supply set associated with log files
supply_cards = ['Cellar','Market','Militia','Mine','Moat','Remodel','Smithy','Village',
                'Woodcutter','Workshop','Curse']

base_cards = ['Gold', 'Silver', 'Copper', 'Estate', 'Duchy', 'Province']

total_card_set = supply_cards + base_cards

#get a list of log files to process
file = open(input_filename, 'r')
log_files= file.readlines()

#tidy up by removing part of path
remove_start = 'gdrive/My Drive/Colab Notebooks/DominionPlayerLogProcessing/'
remove_end = '\n'

for (i,file) in enumerate(log_files):
    log_files[i] = log_files[i][len(remove_start): -len(remove_end)]

print("No of log files: " +str(len(log_files)))

#check for duplicates
set_without_duplicates = set(log_files)
print("No of duplciates: " + str(len(log_files) - len(set_without_duplicates)))

No of log files: 149
No of duplciates: 0


In [28]:
target_directory_for_log_files = 'DominionPlayerLogs/2Player_FG1E_Logs'

In [4]:
#only needs to be run once to extract log files from tar files and store separately

#NOTE: we assume all log files in the input file are in the following
#directory
path = 'DominionPlayerLogs/2010_And_2011'

for log_file in log_files:
    #determine name of tar file to unzip
    substr_path = 'DominionPlayerLogs/2010-all/' #year here is irrelevant just using length, so could 
    #also use 2011-all etc
    tar_file = log_file[len(substr_path): len(substr_path) + 8] + '.tar.bz2'
    
    #determine game-log file needed 
    game_log = log_file[len(substr_path) + 9:]
    
    #next extract tar file
    with tarfile.open(path_string + tar_file, 'r') as tar:
        tar.extract(game_log, target_directory_for_log_files)

NameError: name 'path_string' is not defined

In [29]:
list_of_log_files = os.listdir(target_directory_for_log_files)
#print(list_of_log_files)
print("No of log files: " + str(len(list_of_log_files))) # note one extra compared ot the above due 
#.DS_Store file being picked up

No of log files: 150


In [30]:
#reformat a playtrace into a dataframe
def reformat_playtrace_into_df(deck_by_round, total_card_set, player_scores, player_turns, gameID):
    cols = ['GameID', 'AgentName', 'Player', 'Round', 'TotalRounds', 'FinalScore', 'Win'] + total_card_set
    output = pd.DataFrame(columns = cols)
    row_data = {key: None for key in cols}
    
    player_no = 0
    for player in player_scores.keys():
        for round_ in range(0, len(deck_by_round[player])):
            row_data['GameID'] = gameID
            row_data['AgentName'] = player
            row_data['Player'] = player_no
            row_data['Round'] = round_
            row_data['FinalScore'] = player_scores[player]
            row_data['TotalRounds'] = player_turns[player]
            for player_ in player_scores.keys():
                if player_ != player:
                    if player_scores[player] > player_scores[player_]:
                        row_data['Win'] = 1.0
                    elif player_scores[player] == player_scores[player_]:
                        row_data['Win'] = 0.5
                    else:
                        row_data['Win'] = 0
            for card in total_card_set:
                row_data[card] = deck_by_round[player][round_][card]
            new_row = pd.DataFrame([row_data])
            output = pd.concat([output, new_row], ignore_index = True)
        player_no += 1
    
    output.reset_index()
    return output

#create play trace from cards bought and trashed each turn
def build_play_trace(player_list, total_card_set, incremental_cards_by_turn):
    #need the total number of cards of each type by round for each player
    
    #initialise dictionary
    cards_by_round = {}
    for player in player_list:
        cards_by_round[player] = {}
        for round_no in range(0, len(incremental_cards_by_turn[player])):
            cards_by_round[player][round_no] = {}
            for card in total_card_set:
                cards_by_round[player][round_no][card] = 0
                
    #add starter cards to cards by round
    for player in player_list:
        for round_ in range(0, len(incremental_cards_by_turn[player])):
            cards_by_round[player][round_]['Copper'] = 7
            cards_by_round[player][round_]['Estate'] = 3
          
    #count number of cards of each type for each round
    for player in player_list:
        for round_no in range(0, len(incremental_cards_by_turn[player])):
            #loop over cards for each action type
            for card in incremental_cards_by_turn[player][round_no+1]['buys']:
                #need to add card to this and all subsequent rounds
                for round_ in range(round_no, len(incremental_cards_by_turn[player])):
                    cards_by_round[player][round_][card] += 1
            for card in incremental_cards_by_turn[player][round_no+1]['trashing']:
                for round_ in range(round_no, len(incremental_cards_by_turn[player])):
                    cards_by_round[player][round_][card] -= 1
            for card in incremental_cards_by_turn[player][round_no+1]['trashes']:
                for round_ in range(round_no, len(incremental_cards_by_turn[player])):
                    cards_by_round[player][round_][card] -= 1
            for card in incremental_cards_by_turn[player][round_no+1]['gaining']:
                for round_ in range(round_no, len(incremental_cards_by_turn[player])):
                    cards_by_round[player][round_][card] += 1
                
    return cards_by_round

#extract number of turns for this game for each player
def turns_for_each_player(player_list, html_contents):
    total_turns = []
    count = 0
    for (k,r) in enumerate(html_contents):
        if 'turns' in r.text:
            turn_position = r.text.index('turns')
            #assume number of turns is given by two digits and there is a space to the start of the word 'turn'q
            total_turns.append(int(re.search(r'\d+', r.text[turn_position-3:turn_position]).group()))
            count += 1
            if count == len(player_list):
                break
            
    #note, turns are in same order as player names in html file
    turns_by_player = {}
    count = 0
    for player in player_list:
        turns_by_player[player] = total_turns[count]
        count += 1
    
    return turns_by_player

#following function parses a single line which consists of number of cards (unless equal to one) and card types
#each separated by a colon and ended with a full stop or a horizontal dashed line. It returns a list of cards 
#with card names duplicated according to the number of them in the row. 
#finally, we stop parsing if we find a string that is in the cmd_list that is not equal to the cmd
#this prevents us having issues with statements like 'trashing a Silver and gaining a Gold'
#Also the index of the full stop or dashed line or start of a new command will be returned
def parse_row_into_cards(cmd, cmd_list, html_contents):
    card_list = []
    cmd_list_copy = cmd_list.copy()
    cmd_list_copy.remove(cmd)
    for (k,r) in enumerate(html_contents):
        #Need to be careful that the first content item doesn't contain a '....', causing 
        #the code to pickup a full stop
        #check we aren't finding any other commands
        cmd_bools = [(other_cmd in r) for other_cmd in cmd_list_copy]
        if ((('.' not in r) or (k == 0)) and r.name == None and ('----------------------' not in r) and (not any(cmd_bools))):
            #check to see if trashing text contains the number of cards trashed
            #number is contained in last two digits of text
            try:
                no_cards = int(r[-2:])
            except ValueError:
                no_cards = 1
            #take next entry along which should be the card name
            card_type = r.next_sibling.text
            #and add copies of that to the dictionary
            for count in range(0,no_cards):
                card_list.append(card_type)
        elif (('.' in r) or ('----------------------' in r) or any(cmd_bools)) and (k != 0):
            return (k, card_list)

#function to get players and scores
def players_and_scores(html_soup):
    #look for entries of the form #n <name>: x points' where n is a positive integer - these are tagged with a 'b'
    #in earlier log files the '#n' is dropped and hence we also need to consider this case
    player_scores = {}
    list_b_tags = html_soup.find_all('b'); 
    for tag in list_b_tags:
        s = tag.text
        #extract name (assume string of the form '#n ' comes before the name)
        match = re.search(r'#\d+ ', s)
        if match != None:
            #we need to find the right most ':' (some names may contain a ':')
            colon_index = s.rfind(':')
            if colon_index == -1:
                player = s[match.end():] #sometimes the colon is in the next sibling
            else:
                player = s[match.end():colon_index]
            #next sibing contains the score, assume score is after a string of the form ': ' and is 2 digits long
            score_text = tag.next_sibling
            score = int(re.search(r'-?\d+', score_text[2:5]).group()) 
            player_scores[player] = score
            
    #deal with second case where name is not preceeded by a '#n ', in this case we need to look for a ': n points'
    if len(player_scores) == 0:
        for tag in list_b_tags:
            s = tag.text
            match = re.search(r': (-?\d+) point', s) #we drop the s at the end of points to deal with the case where someone scores 1
            if match != None:
                #in this case the name is contained in the text preceeded by a ':'
                colon_position = match.start()
                if colon_position == 0:
                    #sometime the name is in the previous tag
                    pdb.set_trace()
                    player = s.prev_sibling.text
                else:
                    player = s[:colon_position]
                score = int(re.search(r'-?\d+', s[colon_position+1 : colon_position+4]).group()) 
                player_scores[player] = score
                
    return player_scores

def process_player_actions(player_list, turns_by_player, html_contents):
    #use a dictionary of dictionaries to track gained cards by turn by player, and initialise keys
    incremental_cards_by_turn = {}
    
    #note sometimes a player has one less turn than another player but
    #we still initialise the incremental cards by turn dict to the larger number of turns
    max_no_turns = 0
    for player in player_list:
        if turns_by_player[player] > max_no_turns:
            max_no_turns = turns_by_player[player]
            
    for player in player_list:
        incremental_cards_by_turn[player] = {}
        for turn in range(1, max_no_turns+1):
            incremental_cards_by_turn[player][turn] = {}
            incremental_cards_by_turn[player][turn] = {}
            incremental_cards_by_turn[player][turn] = {}
            incremental_cards_by_turn[player][turn] = {}
        for turn in range(1, max_no_turns+1):
            incremental_cards_by_turn[player][turn]['buys'] = {}
            incremental_cards_by_turn[player][turn]['trashing'] = {}
            incremental_cards_by_turn[player][turn]['gaining'] = {}
            incremental_cards_by_turn[player][turn]['trashes'] = {}
    
    cmd_list = ['buys', 'trashing', 'gaining', 'trashes']
    for player in player_list:
        turn_counter = 1
        #string to check for a buy action
        check_buy = player + ' buys'
        #string to check if current player is trashing a card
        check_trashing = 'trashing'
        #string to check if current player is gaining a card
        check_gaining = 'gaining'
        #string to check if another player trashes a card. Howver, current player can both 'trashes' 
        #and 'trashing'. Also it is possible for a player to trash nothing
        check_player_trashes = [x + ' trashes' for x in player_list] # need to check this!
        #string to check if another player is gains a card (gains is used rather than
        #gaining when a player gets a card out of turn)
        check_player_gains = [ x + ' gains' for x in player_list]
        #string to check end of turn
        check_turn_end = '(' + player + ' draws:'
        for (i,p) in enumerate(html_contents):
            check_turn = player + '\'s' + ' turn ' + str(turn_counter)
            if check_turn in p:
                buy_card_list = []
                trashing_card_list = []
                gaining_card_list = []
                gains_card_list = {} #needs to be a dictionary as multiple players may gain in another player's round
                trashes_card_list = {} #needs to be a dictionary as multiple players may need to trash in another player's round
                for player_ in player_list:
                    gains_card_list[player_] = [] 
                    trashes_card_list[player_] = [] 
                for (j,q) in enumerate(html_contents[i:]):
                    player_gains_list = [ x in q for x in check_player_gains]
                    player_trashes_list = [ x in q for x in check_player_trashes]
                    if check_buy in q:
                        (index, cards) = parse_row_into_cards('buys', cmd_list, html_contents[i + j:])
                        buy_card_list.append(cards)
                    elif check_trashing in q:
                        if 'trashing nothing' in q: #this used to be 'r'?
                            pdb.set_trace()
                            break
                        (index, cards) = parse_row_into_cards('trashing', cmd_list, html_contents[i + j:])
                        trashing_card_list.append(cards)
                    elif check_gaining in q:
                        #also we need to check for a 'gaining nothing' case
                        if 'gaining nothing' in q: #this used to be 'r'?
                            pdb.set_trace()
                            break
                        else:
                            (index, cards) = parse_row_into_cards('gaining', cmd_list, html_contents[i + j:])
                            gaining_card_list.append(cards)
                    elif any(player_gains_list):
                        #next we check if an opponent gains a card, e.g. a curse card
                        #need to loop through opponents
                        for (index, player_gains_check) in enumerate(player_gains_list):
                            if player_gains_check == True:
                                player_ = players_list[index]
                                (index_, cards) = parse_row_into_cards('gains', cmd_list, html_contents[i + j:])
                                gains_card_list[player_].append(cards)
                    elif any(player_trashes_list):
                        #next we check if an opponent trashes a card
                        for (index, player_trashes_check) in enumerate(player_trashes_list):
                            if player_trashes_check == True:
                                player_ = players_list[index]
                                (index_, cards) = parse_row_into_cards('trashes', cmd_list,html_contents[i + j:])
                                trashes_card_list[player_].append(cards)
                    elif check_turn_end in q.text:
                        incremental_cards_by_turn[player][turn_counter]['buys'] = buy_card_list
                        incremental_cards_by_turn[player][turn_counter]['trashing'] = trashing_card_list
                        #pdb.set_trace()
                        incremental_cards_by_turn[player][turn_counter]['gaining'] = gaining_card_list
                        for player_ in gains_card_list.keys():
                            incremental_cards_by_turn[player_][turn_counter]['gains'] = gains_card_list[player_] 
                        for player_ in trashes_card_list.keys():
                            incremental_cards_by_turn[player_][turn_counter]['trashes'] = trashes_card_list[player_] 
                        turn_counter += 1
                        break
    
    #finally we flatten incremental_cards_by_turn into a dictionary of lists
    for player in player_list:
        for turns in range(1, turns_by_player[player]+1):
            for cmd_type in incremental_cards_by_turn[player][turns].keys():
                flattened_list = [item for sublist in incremental_cards_by_turn[player][turns][cmd_type] for item in sublist]
                incremental_cards_by_turn[player][turns][cmd_type] = flattened_list
    
    return incremental_cards_by_turn

In [31]:
#next need to generate a play trace file using data from each log file

#main columns for this file should be GameID, Player name, Player (0 or 1), Round (starting from zero), then list of card names 
#with entries given by the total number of cards a player has in her deck

#set-up log
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='processor.log', mode='a')
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)

gameID = -1 #start counting games from zero
processed_files = []

#files_to_ignore = ['game-20110130-000030-482766c5.html', #ignored as one player buys a curse card
#                   'game-20110213-153439-e3c9416a.html', #ignored as one player buys a curse card
#                   'game-20110117-173643-e788bbed.html'] #ignored as one player buys a curse card

files_to_ignore = []

output = pd.DataFrame()
for game_log in os.listdir(target_directory_for_log_files):
    if game_log not in files_to_ignore:
        if game_log[-4:] == 'html':
            #output file to be processed in log file
            logging.debug("Processing file: " + game_log)

            start_time_for_file = time.time()
            
            gameID+=1

            #store processed file name in a list, just helps with debugging
            processed_files.append(game_log)

            #read and parse html
            file = open(target_directory_for_log_files + '/' + game_log, mode = 'r')
            soup = BeautifulSoup(file, 'html.parser')

            #convert html contents into a list of tabs, navigable strings etc
            pre_tag = soup.find('pre')

            #get player names and scores                      
            player_scores = players_and_scores(soup)

            list_of_players = []
            for player in player_scores.keys():
                list_of_players.append(player)

            #get turns by player
            player_turns = turns_for_each_player(list_of_players, pre_tag.contents)

            incremental_cards_by_turn = process_player_actions(list_of_players, player_turns, pre_tag.contents)

            cards_by_round = build_play_trace(list_of_players, total_card_set, incremental_cards_by_turn)

            #convert play-trace to dataframe
            tmp_df = reformat_playtrace_into_df(cards_by_round, total_card_set, player_scores, player_turns, gameID)

            #add to dataframe for output
            output = pd.concat([output, tmp_df])

#for compatibility with kmeans code we sort and capitalise card names
sorted_output = output.sort_values(by = ['GameID', 'Round']) 
new_cols = ['GameID', 'AgentName', 'Player', 'Round', 'TotalRounds', 'FinalScore', 'Win'] + [card.upper() for card in total_card_set]
sorted_output.columns = new_cols
print(sorted_output) 
#output results file
sorted_output.to_csv(output_filename, sep = ' ', index = False)

   GameID   AgentName Player Round TotalRounds FinalScore  Win CELLAR MARKET  \
0       0  Silvermink      0     0          17         33  1.0      0      0   
17      0     Draggor      1     0          16         21    0      1      0   
1       0  Silvermink      0     1          17         33  1.0      1      0   
18      0     Draggor      1     1          16         21    0      1      1   
2       0  Silvermink      0     2          17         33  1.0      1      0   
..    ...         ...    ...   ...         ...        ...  ...    ...    ...   
31    148     mazungu      1    14          16         25    0      1      2   
15    148     mazgirl      0    15          17         31  1.0      1      1   
32    148     mazungu      1    15          16         25    0      1      2   
16    148     mazgirl      0    16          17         31  1.0      1      1   
33    148     mazungu      1    16          16         25    0      1      2   

   MILITIA  ... VILLAGE WOODCUTTER WORK